In [43]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('summary_data.csv')

In [3]:
pd.set_option('max_columns', None)

In [6]:
data.head()

,file,line,start,end,speaker,speaker_role,word_count,duration,text,start_idx,end_idx,gs_score,pitch_log_diff_variance,pitch_log_mean,pitch_log_stdev,pitch_log_2pct,pitch_log_25pct,pitch_log_50pct,pitch_log_75pct,pitch_log_98pct,onset_count,onset_str_mean,onset_str_stddev,onset_str_entropy,onset_time_diff_mean,onset_time_diff_stddev,onset_time_diff_entropy,word_rate,onset_rate,year,justice_word_count_mean,justice_word_count_std,justice_duration_mean,justice_duration_std,justice_gs_score_mean,justice_gs_score_std,justice_pitch_log_diff_variance_mean,justice_pitch_log_diff_variance_std,justice_pitch_log_mean_mean,justice_pitch_log_mean_std,justice_pitch_log_stdev_mean,justice_pitch_log_stdev_std,justice_pitch_log_2pct_mean,justice_pitch_log_2pct_std,justice_pitch_log_25pct_mean,justice_pitch_log_25pct_std,justice_pitch_log_50pct_mean,justice_pitch_log_50pct_std,justice_pitch_log_75pct_mean,justice_pitch_log_75pct_std,justice_pitch_log_98pct_mean,justice_pitch_log_98pct_std,justice_onset_count_mean,justice_onset_count_std,justice_onset_str_mean_mean,justice_onset_str_mean_std,justice_onset_str_stddev_mean,justice_onset_str_stddev_std,justice_onset_str_entropy_mean,justice_onset_str_entropy_std,justice_onset_time_diff_mean_mean,justice_onset_time_diff_mean_std,justice_onset_time_diff_stddev_mean,justice_onset_time_diff_stddev_std,justice_onset_time_diff_entropy_mean,justice_onset_time_diff_entropy_std,justice_word_rate_mean,justice_word_rate_std,justice_onset_rate_mean,justice_onset_rate_std,justice_year_word_count_mean,justice_year_word_count_std,justice_year_duration_mean,justice_year_duration_std,justice_year_gs_score_mean,justice_year_gs_score_std,justice_year_pitch_log_diff_variance_mean,justice_year_pitch_log_diff_variance_std,justice_year_pitch_log_mean_mean,justice_year_pitch_log_mean_std,justice_year_pitch_log_stdev_mean,justice_year_pitch_log_stdev_std,justice_year_pitch_log_2pct_mean,justice_year_pitch_log_2pct_std,justice_year_pitch_log_25pct_mean,justice_year_pitch_log_25pct_std,justice_year_pitch_log_50pct_mean,justice_year_pitch_log_50pct_std,justice_year_pitch_log_75pct_mean,justice_year_pitch_log_75pct_std,justice_year_pitch_log_98pct_mean,justice_year_pitch_log_98pct_std,justice_year_onset_count_mean,justice_year_onset_count_std,justice_year_onset_str_mean_mean,justice_year_onset_str_mean_std,justice_year_onset_str_stddev_mean,justice_year_onset_str_stddev_std,justice_year_onset_str_entropy_mean,justice_year_onset_str_entropy_std,justice_year_onset_time_diff_mean_mean,justice_year_onset_time_diff_mean_std,justice_year_onset_time_diff_stddev_mean,justice_year_onset_time_diff_stddev_std,justice_year_onset_time_diff_entropy_mean,justice_year_onset_time_diff_entropy_std,justice_year_word_rate_mean,justice_year_word_rate_std,justice_year_onset_rate_mean,justice_year_onset_rate_std
0,11-681,4,62.906,82.218,Ruth_Bader_Ginsburg,scotus_justice,45,19.312,But how does it differ from the typical bargai...,1006496,1315488,-1.173626,4.420750,5.279096,3.690280,0.0,0.000000,7.710526,7.864693,8.272526,116.0,0.784787,1.376932,-3.093341,0.227200,0.214818,-79.000485,2.330157,6.006628,2013.0,65.111683,23.936758,30.546159,12.900663,-0.120233,0.619828,4.332276,0.771608,4.374778,0.845197,3.732496,0.182046,0.0,0.0,0.204550,1.208909,5.598532,3.247815,7.666009,0.706801,8.215027,0.137034,171.511389,61.861765,0.682518,0.075204,1.139276,0.156244,-3.898827,1.008391,0.238596,0.049807,0.256829,0.069155,-63.762414,36.458768,2.200660,0.373876,5.771207,0.902757,68.093496,23.457547,32.766317,12.892902,-0.195107,0.624182,4.312981,0.726183,4.787263,0.597654,3.698301,0.180863,0.0,0.0,0.301190,1.466223,7.070379,1.639869,7.784569,0.138150,8.266880,0.125538,186.662602,66.358980,0.701429,0.065423,1.160205,0.122135,-3.589175,0.825202,0.238431,0.043301,0.251816,0.063445,-62.786753,38.406475,2.141787,0.350414,5.837687,0.860461
1,11-681,6,99.497,119.410,Sonia_Sotomayor,scotus_justice,40,19.913,Is your argument dependent on this being sort ...,1591952,1910560,-0

In [7]:
df = data.copy()

In [8]:
df.columns

Index(['file', 'line', 'start', 'end', 'speaker', 'speaker_role', 'word_count',
       'duration', 'text', 'start_idx',
       ...
       'justice_year_onset_time_diff_mean_mean',
       'justice_year_onset_time_diff_mean_std',
       'justice_year_onset_time_diff_stddev_mean',
       'justice_year_onset_time_diff_stddev_std',
       'justice_year_onset_time_diff_entropy_mean',
       'justice_year_onset_time_diff_entropy_std',
       'justice_year_word_rate_mean', 'justice_year_word_rate_std',
       'justice_year_onset_rate_mean', 'justice_year_onset_rate_std'],
      dtype='object', length=110)

In [9]:
df.drop(['file', 'line', 'start', 'end', 'speaker', 'speaker_role', 'text', 
         'start_idx','end_idx'], axis=1, inplace=True)





In [10]:
#set half of score as grandstanding for now to keep balanced data

df['Grandstanding_ind'] = np.where(df['gs_score']>data['gs_score'].mean(),1,0)


In [11]:
df.drop('gs_score', axis=1, inplace=True)

In [35]:
df = df.dropna()

In [36]:
X = df.loc[:, df.columns != 'Grandstanding_ind']
y = df.loc[:, df.columns == 'Grandstanding_ind']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=117)


In [38]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,  
    test_size=0.2, random_state=117)

In [41]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Learning rate:  0.05
Accuracy score (training): 0.629
Accuracy score (validation): 0.621
Learning rate:  0.075
Accuracy score (training): 0.625
Accuracy score (validation): 0.620
Learning rate:  0.1
Accuracy score (training): 0.626
Accuracy score (validation): 0.622


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Learning rate:  0.25
Accuracy score (training): 0.639
Accuracy score (validation): 0.628
Learning rate:  0.5
Accuracy score (training): 0.675
Accuracy score (validation): 0.658
Learning rate:  0.75
Accuracy score (training): 0.687
Accuracy score (validation): 0.658
Learning rate:  1
Accuracy score (training): 0.680
Accuracy score (validation): 0.652


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [55]:
n = [20, 50, 100, 200, 500, 1000]

for i in n:
    gb_clf = GradientBoostingClassifier(n_estimators=i, learning_rate=0.75, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("n_estimators: ", i)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


n_estimators:  20
Accuracy score (training): 0.687
Accuracy score (validation): 0.658
n_estimators:  50
Accuracy score (training): 0.706
Accuracy score (validation): 0.679


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


n_estimators:  100
Accuracy score (training): 0.727
Accuracy score (validation): 0.686


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


n_estimators:  200
Accuracy score (training): 0.741
Accuracy score (validation): 0.689


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


n_estimators:  500
Accuracy score (training): 0.768
Accuracy score (validation): 0.679


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


n_estimators:  1000
Accuracy score (training): 0.802
Accuracy score (validation): 0.678


In [56]:
max_fea = [2, 5, 10, 20]

for i in max_fea:
    gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.75, max_features=i, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("max_features: ", i)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_features:  2
Accuracy score (training): 0.741
Accuracy score (validation): 0.689


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_features:  5
Accuracy score (training): 0.758
Accuracy score (validation): 0.675


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_features:  10
Accuracy score (training): 0.768
Accuracy score (validation): 0.678


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_features:  20
Accuracy score (training): 0.778
Accuracy score (validation): 0.671


In [57]:
max_dep = [2, 5, 10, 20]

for i in max_dep:
    gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.75, max_features=10, max_depth=i, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("max_depth: ", i)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_depth:  2
Accuracy score (training): 0.768
Accuracy score (validation): 0.678


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_depth:  5
Accuracy score (training): 0.991
Accuracy score (validation): 0.655


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_depth:  10
Accuracy score (training): 1.000
Accuracy score (validation): 0.666


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


max_depth:  20
Accuracy score (training): 1.000
Accuracy score (validation): 0.599


In [59]:
gb_clf2 = GradientBoostingClassifier(n_estimators=200, learning_rate=0.75, max_features=10, max_depth=2, random_state=117)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_val)

print("Confusion Matrix:")
print(confusion_matrix(y_val, predictions))

print("Classification Report")
print(classification_report(y_val, predictions))

predictions2 = gb_clf2.predict(X_test)

print("Confusion Matrix - test:")
print(confusion_matrix(y_test, predictions2))

print("Classification Report")
print(classification_report(y_test, predictions2))




/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Confusion Matrix:
[[ 768  466]
 [ 441 1055]]
Classification Report
              precision    recall  f1-score   support

           0       0.64      0.62      0.63      1234
           1       0.69      0.71      0.70      1496

    accuracy                           0.67      2730
   macro avg       0.66      0.66      0.66      2730
weighted avg       0.67      0.67      0.67      2730

Confusion Matrix - test:
[[1020  526]
 [ 543 1324]]
Classification Report
              precision    recall  f1-score   support

           0       0.65      0.66      0.66      1546
           1       0.72      0.71      0.71      1867

    accuracy                           0.69      3413
   macro avg       0.68      0.68      0.68      3413
weighted avg       0.69      0.69      0.69      3413

